In [ ]:
pip install facebook_business

In [ ]:
pip install pandas

In [ ]:
import time
import pandas as pd
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.business import Business
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.adreportrun import AdReportRun

# In case of error code 1 with message "Please reduce the amount of data you're asking for, then retry your request"
def wait_for_async_job(request, timeout=300):
    for _ in range(timeout):
        time.sleep(3)
        job = request.api_get()
        status = job[AdReportRun.Field.async_status]
        print(status)
        if status == "Job Completed":
            return job.get_result()

def get_data(accounts, fields, params):

    res_list = []
    f_acc_list = []

    # Insights
    for account in accounts:
        acc = AdAccount(account['id'])
        try:
            res = acc.get_insights(fields=fields, params=params)
            if res:
                res_list += [dict(item) for item in res]
        except Exception as e:
            print(f"This account got an error: {e}")
            f_acc_list.append(acc)

    if f_acc_list:
        # Async Insights
        for acc in f_acc_list:
            try:
                request = acc.get_insights(fields=fields, params=params, is_async=True)
                async_res = wait_for_async_job(request)
                if async_res:
                    res_list += [dict(item) for item in async_res]
            except Exception as e:
                print(f"Error in: {e}")
    return res_list

def clean_export(res_list):
    df = pd.json_normalize(res_list)
    df.to_csv('facebook_data.csv', index=False, encoding='utf-8-sig')

def call_api():
    fields = [
            AdsInsights.Field.account_id,
            AdsInsights.Field.account_name,
            AdsInsights.Field.campaign_id,
            AdsInsights.Field.campaign_name,
            AdsInsights.Field.objective,
            AdsInsights.Field.creative_media_type,
            AdsInsights.Field.impressions,
            AdsInsights.Field.spend,
            AdsInsights.Field.inline_link_clicks
            ]

    params = {
            'date_preset': 'today',
            'level': 'campaign'
            }

    my_app_id = 'your_app_id'
    my_app_secret = 'your_app_secret'
    my_access_token = 'your_access_token'
    FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

    business_id = Business('your_business_id')

    accounts = business_id.get_owned_ad_accounts()
    data = get_data(accounts, fields, params)
    clean_export(data)

if __name__ == '__main__':
    call_api()